In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [10]:
# Cargar el archivo Excel
df = pd.read_csv('df_limpio.csv')

X_dummies = df[[
    'FAMI_TIENELAVADORA_Si','FAMI_TIENEAUTOMOVIL_Si','FAMI_TIENECOMPUTADOR_Si',
]]
X_categoricas = df[[
    'ESTU_COD_RESIDE_MCPIO','FAMI_PERSONASHOGAR','ESTU_COD_MCPIO_PRESENTACION','ESTU_TIPODOCUMENTO','FAMI_CUARTOSHOGAR','FAMI_EDUCACIONMADRE','FAMI_ESTRATOVIVIENDA','COLE_AREA_UBICACION_URBANO'
]]
X_categoricas_dummified = pd.get_dummies(X_categoricas, drop_first=True)

X = pd.concat([X_dummies, X_categoricas_dummified], axis=1)

In [11]:
# Prepara tus datos (reemplaza con tu df real)
y = df['ALTO_RENDIMIENTO_MATE']  

In [12]:
# Codificación de variable objetivo
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

In [13]:
# Escalamiento y split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)

In [14]:
# Espacios de hiperparámetros
param_grid = {
    'hidden_layers': [1, 2],
    'neurons': [32, 64],
    'learning_rate': [0.001, 0.01],
    'activation': ['relu', 'tanh'],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

In [15]:
# Entrenamiento con búsqueda
resultados = []

# Calcular número total de combinaciones
total_iteraciones = (
    len(param_grid['hidden_layers']) *
    len(param_grid['neurons']) *
    len(param_grid['learning_rate']) *
    len(param_grid['activation']) *
    len(param_grid['batch_size']) *
    len(param_grid['epochs'])
)

# Inicializar contador
iteracion = 1

for hl in param_grid['hidden_layers']:
    for neurons in param_grid['neurons']:
        for lr in param_grid['learning_rate']:
            for activation in param_grid['activation']:
                for batch in param_grid['batch_size']:
                    for ep in param_grid['epochs']:

                        print(f"Iteración {iteracion} de {total_iteraciones}...", end='\r')

                        model = Sequential()
                        model.add(Dense(neurons, activation=activation, input_shape=(X_train.shape[1],)))
                        for _ in range(hl - 1):
                            model.add(Dense(neurons, activation=activation))
                        model.add(Dense(y_cat.shape[1], activation='softmax'))

                        model.compile(optimizer=Adam(learning_rate=lr),
                                      loss='categorical_crossentropy',
                                      metrics=['accuracy'])

                        history = model.fit(X_train, y_train, epochs=ep,
                                            batch_size=batch, validation_split=0.2,
                                            verbose=0)

                        acc = history.history['val_accuracy'][-1]

                        resultados.append({
                            'Capas ocultas': hl,
                            'Neuronas': neurons,
                            'LR': lr,
                            'Activación': activation,
                            'Batch Size': batch,
                            'Epochs': ep,
                            'Val Accuracy': round(acc, 4)
                        })

                        iteracion += 1

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Mostrar resultados ordenados
resultados_df = pd.DataFrame(resultados).sort_values(by='Val Accuracy', ascending=False)
print(resultados_df.head(10))

   Capas ocultas  Neuronas     LR Activación  Batch Size  Epochs  Val Accuracy
0              1        32  0.001       relu          32      50        0.7034
1              1        32  0.001       relu          32     100        0.7034
2              1        32  0.001       relu          64      50        0.7034
3              1        32  0.001       relu          64     100        0.7034
4              1        32  0.001       tanh          32      50        0.7034
5              1        32  0.001       tanh          32     100        0.7034
6              1        32  0.001       tanh          64      50        0.7034
7              1        32  0.001       tanh          64     100        0.7034
8              1        32  0.010       relu          32      50        0.7034
9              1        32  0.010       relu          32     100        0.7034
